# Example on how load and save from Hugging Face Hub

[Source](https://colab.research.google.com/drive/1Uqy8i6G4og6awtNmJGatqek2mgBzvvXu?usp=sharing#scrollTo=F7ouGeDwYzZt)

Author: [Pavel Iakubovskii](https://www.linkedin.com/in/pavel-iakubovskii/)

In [ ]:
from huggingface_hub import notebook_login

/opt/homebrew/Caskroom/miniconda/base/envs/albumentations_examples/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
notebook_login()

ImportError: The `notebook_login` function can only be used in a notebook (Jupyter or Colab) and you need the `ipywidgets` module: `pip install ipywidgets`.

In [ ]:
import albumentations as A

transform = A.Compose(
    [
        A.RandomCrop(256, 256),
        A.HorizontalFlip(),
        A.RandomBrightnessContrast(),
        A.RGBShift(),
        A.Normalize(),
    ],
    seed=137,
    strict=True,
)

evaluation_transform = A.Compose(
    [
        A.PadIfNeeded(256, 256),
        A.Normalize(),
    ],
    seed=137,
    strict=True,
)

transform.save_pretrained("qubvel-hf/albu", key="train")
# ^ this will save the transform to a directory "qubvel-hf/albu" with filename "albumentations_config_train.json"

transform.save_pretrained("qubvel-hf/albu", key="train", push_to_hub=True)
# ^ this will save the transform to a directory "qubvel-hf/albu" with filename "albumentations_config_train.json"
# + push the transform to the Hub to the repository "qubvel-hf/albu"

transform.push_to_hub("qubvel-hf/albu", key="train")
# ^ this will push the transform to the Hub to the repository "qubvel-hf/albu" (without saving it locally)

loaded_transform = A.Compose.from_pretrained("qubvel-hf/albu", key="train")
# ^ this will load the transform from local folder if exist or from the Hub repository "qubvel-hf/albu"

evaluation_transform.save_pretrained("qubvel-hf/albu", key="eval", push_to_hub=True)
# ^ this will save the transform to a directory "qubvel-hf/albu" with filename "albumentations_config_eval.json"

loaded_evaluation_transform = A.Compose.from_pretrained("qubvel-hf/albu", key="eval")
# ^ this will load the transform from the Hub repository "qubvel-hf/albu"

In [ ]:
# check
import numpy as np

image = np.random.randint(0, 255, (100, 200, 3), dtype=np.uint8)

preprocessed_image_1 = evaluation_transform(image=image)["image"]
preprocessed_image_2 = loaded_evaluation_transform(image=image)["image"]

assert np.allclose(preprocessed_image_1, preprocessed_image_2)